In [1]:
import json
import os
import openai

with open('keys.json', "r") as f:
    keys = json.load(f)
print(keys.keys())

openai.api_key = keys['secret']

dataDir = "datasets/"
with open(dataDir + "counterfact.json", "r") as f:
    counterfact = json.load(f)
if os.path.exists(dataDir + "related.json"):
    with open(dataDir +"related.json", "r") as f:
        related = json.load(f)
else:
    related = []
print(len(counterfact))
print(len(related))
totalTokens = 0

def costEstimate(tokens):
    return (tokens/1000) * 0.002


dict_keys(['secret'])


In [ ]:
def genMsg(idx):
    msg = counterfact[idx]['requested_rewrite']['prompt']
    msg = msg.replace("{}", counterfact[idx]['requested_rewrite']['subject'])
    msg = msg + " " + counterfact[idx]['requested_rewrite']['target_true']["str"] + "."
    msg = msg + f" Can you write a list on one line without commas of related words to the subject of the first sentence, {counterfact[idx]['requested_rewrite']['subject']}."
    return msg

In [ ]:
def genResponse(msg):
    global totalTokens
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": msg}
        ],
        max_tokens=20,
        # stop=["\n", "."],
        temperature=0.8,
        presence_penalty=0.0,
    )
    totalTokens += response["usage"]["total_tokens"]
    return response

In [ ]:
punctuation = [".", ",", "!", "?", ";", ":", "-", "(", ")", "[", "]", "{", "}", "<", ">"]
def postProcess(words):
    for punc in punctuation:
        words = words.replace(punc, " ")
    words = words.split(" ")
    words = [word for word in words if word != ""]
    return words

In [44]:
i = len(related)
print(counterfact[i])
msg = genMsg(i)
print(msg)

IndexError: list index out of range

In [ ]:
# response = genResponse(msg)

In [ ]:
# print(response["choices"][0]["message"]["content"])
# print(postProcess(response["choices"][0]["message"]["content"]))
# print(response["choices"][0]["finish_reason"])
# print(response["usage"]["total_tokens"])
# print(costEstimate(totalTokens))

In [8]:
testing = False

for i in range(len(related), len(counterfact)):
    msg = genMsg(i)
    response = genResponse(msg)
    words = postProcess(response["choices"][0]["message"]["content"])
    if testing:
        print(msg)
        print(response["choices"][0]["message"]["content"])
        print(words)

    entry = counterfact[i]
    entry["relatedWords"] = words
    entry["responseRaw"] = response.to_dict()
    related.append(entry)
    with open(dataDir + "related.json", "w") as f:
        json.dump(related, f)

    # print(response["choices"][0]["finish_reason"])
    print(f"{i} / {len(counterfact)} Current Cost {costEstimate(totalTokens)}", end="\r")
    if costEstimate(totalTokens) > 10:
        break

    if testing:
        break

In [6]:
print(related[-1].keys())
print(related[-1]["relatedWords"])
print(len(related))

dict_keys(['case_id', 'pararel_idx', 'requested_rewrite', 'paraphrase_prompts', 'neighborhood_prompts', 'attribute_prompts', 'generation_prompts', 'relatedWords', 'responseRaw'])
['actor', 'theatre', 'film', 'stage', 'performance', 'drama', 'comedy', 'tragedy', 'audition', 'rehearsal', 'script', 'character', 'role', 'director', 'producer', 'cinematographer', 'screenplay', 'set', 'design']
21919


In [1]:
from collections import deque
import json
import os
import numpy as np
from torch.utils.data.dataset import IterableDataset, Dataset
from torch.utils.data import DataLoader
import torch as th
from torchinfo import summary
from transformers import DataCollatorForLanguageModeling
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoConfig

from core import padded_stack, stack_dicts_list

In [2]:
dataDir = "datasets/"
with open(dataDir +"related.json", "r") as f:
    relatedJson = json.load(f)

print(len(relatedJson))

21919


In [25]:
# json with strings to tuple of tensors
class RelatedDataset(Dataset):
    def __init__(self, dataJson, tokenizer, pad_to_multiple_of=1):
        self.dataJson = dataJson
        self.tokenizer = tokenizer
        self.pad_token_id = tokenizer.pad_token_id

        self.maxLenPrompt = 0
        self.maxLenResponse = 0
        self.maxLenRelated = 0
        self.pad_to_multiple_of = pad_to_multiple_of

    def __len__(self):
        return len(self.dataJson)

    def __getitem__(self, index):
        prompt, response, related = self.formatText(index)

        promptT = self.tokenizer(prompt, return_tensors="pt")["input_ids"].squeeze(0)
        responseT = self.tokenizer(response, return_tensors="pt")["input_ids"].squeeze(0)
        relatedT = self.tokenizer(related, return_tensors="pt")["input_ids"].squeeze(0)

        return promptT, responseT, relatedT

    def formatText(self, idx):
        # print("data index", idx)
        prompt = self.dataJson[idx]['requested_rewrite']['prompt']
        prompt = prompt.replace("{}", self.dataJson[idx]['requested_rewrite']['subject'])
        response = self.dataJson[idx]['requested_rewrite']['target_true']["str"] + "."
        related = self.dataJson[idx]['relatedWords']
        relatedStr = related[0]
        for r in related[1:]:
            relatedStr += " " + r

        relatedStr = relatedStr.replace(".","")
        relatedStr = relatedStr.replace(",","")
        relatedStr = relatedStr.replace("  "," ")

        return prompt, response, relatedStr
    
# batching and padding, use left pad tokkenizer
class RelatedCollator():
    def __init__(self, tokenizer, pad_to_multiple_of=1):
        self.tokenizer = tokenizer
        self.textCollator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, pad_to_multiple_of=pad_to_multiple_of)
        self.pad_token_id = tokenizer.pad_token_id

    def __call__(self, batch):
        prompt, response, related = zip(*batch)

        # print(prompt)
        self.tokenizer.padding_side = "left"
        prompt = self.textCollator(prompt)
        self.tokenizer.padding_side = "right"
        response = self.textCollator(response)
        self.tokenizer.padding_side = "left"
        related = self.textCollator(related)

        # causing index out of range error
        del prompt["labels"]
        del response["labels"]
        del related["labels"]

        prompt["attention_mask"] = th.where(prompt["input_ids"] == self.pad_token_id, 0, 1)
        response["attention_mask"] = th.where(response["input_ids"] == self.pad_token_id, 0, 1)
        related["attention_mask"] = th.where(related["input_ids"] == self.pad_token_id, 0, 1)

        return prompt, response, related # token index in ["input_ids"], -100 for pad in ["labels"]

In [26]:
# left pad so all generations start at the same index
tokenizer = AutoTokenizer.from_pretrained("gpt2", padding_side='left', return_special_tokens_mask=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
print(tokenizer.pad_token_id)
dataset = RelatedDataset(relatedJson, tokenizer)
collator = RelatedCollator(tokenizer, dataset.pad_to_multiple_of)
dataloader = DataLoader(dataset, batch_size=2, collate_fn=collator, shuffle=True)

50256


In [27]:
# test dataloader
maxLenPrompt = 0
maxLenResponse = 0
maxLenRelated = 0

for i in range(len(dataset)):
    prompt, response, related = dataset[i]
    maxLenPrompt = max(maxLenPrompt, prompt.shape[0])
    maxLenResponse = max(maxLenResponse, response.shape[0])
    maxLenRelated = max(maxLenRelated, related.shape[0])
    break

print(maxLenPrompt)
print(maxLenResponse)
print(maxLenRelated)

dataset.maxLenPrompt = maxLenPrompt
dataset.maxLenResponse = maxLenResponse
dataset.maxLenRelated = maxLenRelated

9
2
20


In [28]:
collator = RelatedCollator(tokenizer, dataset.pad_to_multiple_of)
dl = DataLoader(dataset, batch_size=2, collate_fn=collator, shuffle=True)

In [29]:
for prompt, response, related in dl:
    print("prompt", prompt)
    print("reponse", response)
    print("related", related)
    break

prompt {'input_ids': tensor([[32117, 44414,   280,    11, 10834,   286],
        [   33,   363, 25791,   293,   318,   287]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1]])}
reponse {'input_ids': tensor([[24111,    13, 50256],
        [16748,  5439,    13]]), 'attention_mask': tensor([[1, 1, 0],
        [1, 1, 1]])}
related {'input_ids': tensor([[50256, 50256, 50256, 50256, 50256, 50256, 50256, 34470,  6749, 28278,
          6853,  6260, 18026, 18026],
        [16748,  5439,  7072,  3734, 17423, 22158, 33072, 27407,  2815,  3491,
         15162, 24800, 40250,  1570]]), 'attention_mask': tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [30]:
from tuned_lens.nn.lenses import TunedLens, LogitLens
from transformers import AutoModelForCausalLM, AutoTokenizer

device = th.device('cuda')
loadModel = False
modelName = 'gpt2'
# modelName = 'sshleifer/tiny-gpt2'
# To try a diffrent modle / lens check if the lens is avalible then modify this code
model = AutoModelForCausalLM.from_pretrained(modelName)
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained(modelName)
if loadModel:
    tuned_lens = TunedLens.load(modelName, map_location=device)
else:
    tuned_lens = TunedLens(model)
tuned_lens = tuned_lens.to(device)
logit_lens = LogitLens(model)

In [31]:
import os
windows = False
try:
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  if os.name == 'nt':
    windows = True

In [32]:
from torchinfo import summary
print(summary(model))
print(summary(tuned_lens))

Layer (type:depth-idx)                             Param #
GPT2LMHeadModel                                    --
├─GPT2Model: 1-1                                   --
│    └─Embedding: 2-1                              38,597,376
│    └─Embedding: 2-2                              786,432
│    └─Dropout: 2-3                                --
│    └─ModuleList: 2-4                             --
│    │    └─GPT2Block: 3-1                         7,087,872
│    │    └─GPT2Block: 3-2                         7,087,872
│    │    └─GPT2Block: 3-3                         7,087,872
│    │    └─GPT2Block: 3-4                         7,087,872
│    │    └─GPT2Block: 3-5                         7,087,872
│    │    └─GPT2Block: 3-6                         7,087,872
│    │    └─GPT2Block: 3-7                         7,087,872
│    │    └─GPT2Block: 3-8                         7,087,872
│    │    └─GPT2Block: 3-9                         7,087,872
│    │    └─GPT2Block: 3-10                        7,08

In [34]:
from tuned_lens.utils import (
    maybe_all_reduce,
    shift_labels,
    shift_preds,
    send_to_device,
)

from tuned_lens.residual_stream import ResidualStream, record_residual_stream
with th.no_grad():
    for batch_t in dl:
        batch_t = send_to_device(batch_t, device)
        # batch is left pad, response is right pad, related is left pad
        # _ _ _ _ prompt | response _ _ _
        batch, response, related = batch_t
        print("prompt", batch)
        print("reponse", response)
        print("related", related)

        print(batch["input_ids"].shape)
        print(batch["attention_mask"].shape)
        
        
        with record_residual_stream(model) as stream:
            output = model(**batch)
            print(dict(output).keys())
        
        print(response["input_ids"].shape)
        print(response["attention_mask"].shape)
        
        # needs an attention mask for full prompt, but input ids dont include past keys, logits are just for response tokens
        full_att = th.cat([batch["attention_mask"], response["attention_mask"]], dim=1)
        responseOutput = model(input_ids=response["input_ids"], attention_mask=full_att, past_key_values=output.past_key_values)

        print(dict(responseOutput).keys())
        print(full_att.shape)
        break

prompt {'input_ids': tensor([[38743,  5326,    11,   508,  5341,   262,  2292],
        [50256, 50256, 50256, 34979, 46658, 20973,   287]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 1, 1, 1, 1]], device='cuda:0')}
reponse {'input_ids': tensor([[24385,  1891,    13],
        [31710,    13, 50256]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1],
        [1, 1, 0]], device='cuda:0')}
related {'input_ids': tensor([[15914, 16076,  3554,  3714,   263,  1074,  2137, 10242, 30286,  6737,
         24955,  2214],
        [50256, 31710,  2647,  4097,  2854, 32010,  3223, 16212,  8984, 28201,
         23689, 46644]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
torch.Size([2, 7])
torch.Size([2, 7])
dict_keys(['logits', 'past_key_values'])
torch.Size([2, 3])
torch.Size([2, 3])
dict_keys(['logits', 'past_key_values'])
torch.Size([2, 10])
